In [125]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pymc3 as pm

# MCMC

Knowing the process of MCMC gives you insight into whether your algorithm has converged

When we set up a Bayesian inference problem with N-unknowns, we are implicitly creating an N-dimensional space for the prior distributions to exist in. Associated with the space is an additional dimension, which we can describe as the surface, or curve, that sits on top of the space, that reflects the prior probability of a particular point
 - The surface on the space is defined by our prior distributions

In [21]:
x, y = np.linspace(0, 5, 100), np.linspace(0, 5, 100)
X, Y = np.meshgrid(x, y)

# Exponential distribution - priors
exp_x = stats.expon.pdf(x, scale=5)
exp_y = stats.expon.pdf(x, scale=10)

M = np.dot(exp_x[:, None], exp_y[None, :])

<Figure size 432x288 with 0 Axes>

In [24]:
fig = px.imshow(M, origin='lower')
fig.update_layout(title='Prior Distribution Space')
fig.show()

In [23]:
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=M)])
fig.update_layout(title='Prior Probability of a "point"',
                  width=750, height=500)
fig.show()

If these surfaces describe our prior distributions on the unknowns, what happens to our space after we incorporate our observed data $X$? 

The data $X$ does **not change the space**, but it **changes the surface of the space** by pulling and stretching the fabric of the prior surface to reflect where the true parameters are likely to be
 - More data means more pulling and stretching, and our original surface may become mangled or insignificant compared to the newly formed surface
 - Less data, and our original shape is more present

Regardless, the **resulting surface describes the new posterior distribution**

The tendency of the observed data to _push up_ the posterior probability in certain areas is checked by the prior probability distribution such that if the prior probability is less for that value, then this means more resistance
 - If the prior probability states a low probability of an outcome in the event space such that their is more resistance near that value

# Exploring the landscape using MCMC

We should explore the posterior space generated by our prior and observed data to find the posterior mountain. However, searching an $N$-dimensional space is exponentially difficult in $N$ (the curse of dimensionality). The size of the space quickly blows up as we increase $N$

The idea behind MCMC is to perform an intelligent search of this space. Search implies we are looking for the most probable range of values (parameters) in our landscape

MCMC returns samples (traces) from the posterior distribution, not the distribution itself. MCMC performs a task similar to repeatedly asking “How likely is this pebble I found to be from the mountain I am searching for?
 - Returning these traces from the posterior distribution, we hope that MCMC will converge toward the areas of posterior probability. MCMC does this by exploring nearby positions and moving into areas with the highest probability. 

Converging usually implies moving forward toward a point in space, but MCMC moves toward a broad area in the space and randomly walks around that area, picking up samples from that area. 

**Objectives**: 

 - First, we will do a Bayesian analysis where we compute the posterior due to low dimensionality
 - Second, we will approximate the posterior that we have computed with MCMC in PyMC3 and with variational inference due to "high dimensionality"

Example:

Say you have a manufacturing plant, and at a certain step, a device is measuring the weight of the good(s). We have observed two measurements of the weight's good to be [92, 105]. The weight measurement is subject to some error that is Gaussian distributed with mean $\mu$ and variance $\sigma^2$

Thus, we can define our likelihood function as $P(D | \theta) = P(D | \mu, \sigma)$
 - "What is the probability of observing the data given these parameters?"

## Likelihood Function

In [110]:
# Observed data
weights = np.array([92, 105])

# First, define the grid of values for mu and sigma
mu = np.linspace(50, 150)
sigma = np.linspace(0, 15)

mm, ss = np.meshgrid(mu, sigma)

# Likelihood computation
likelihood = stats.norm(mm, ss).pdf(weights[0]) * stats.norm(mm, ss).pdf(weights[1])

In [117]:
fig = px.imshow(likelihood, x=mu, y=sigma, origin='lower', aspect='auto')
fig.show()

## Prior Distribution

Besides the likelihood function, Baye's lets us incorporate prior beliefs into estimating the parameters. Maybe from prior production experience, we know our products typically weight, on average, about 70 pounds and thus follow a gaussian distribution. Then, for the variance, I believe this follows a Cauchy distribution:

$$\mu \sim N(70, 10^2)$$

$$\sigma \sim Cauchy(0, 10^2)$$



In [119]:
# mm = mean and ss = sigma
prior = stats.norm(70, 10).pdf(mm) * stats.cauchy(0, 10).pdf(ss)

In [118]:
fig = px.imshow(prior, x=mu, y=sigma, origin='lower', aspect='auto')
fig.show()

## Posterior Distribution

As this example is only in 2d, we can apply Bayes formula to compute the posterior directly. As you increase the amount of dimensions or define a more complex model, the calculation of $P(D)$ becomes intractable

In [122]:
posterior = prior * likelihood

fig = px.imshow(posterior, x=mu, y=sigma, origin='lower', aspect='auto')
fig.show()

## Addressing Intractability with MCMC



In [80]:
with pm.Model():

    # Prior distribution
    mu = pm.Normal('mu', mu=100, sd=10)
    sigma = pm.HalfCauchy('sigma', 10)

    # Likelihood
    observed = pm.Normal('observed', mu=mu, sigma=sigma, observed=weights)

    # Sampling
    trace = pm.sample(draws=10000, chains=1)

/Users/gabestechschulte/opt/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:468: FutureWarning:

In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sigma, mu]


Sampling 1 chain for 1_000 tune and 10_000 draw iterations (1_000 + 10_000 draws total) took 15 seconds.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [135]:
fig = px.scatter(x=trace['mu'], y=trace['sigma'], color_continuous_scale=True)
fig.show()

## Markov Chains and Metropolis-Hastings

First, an example involving mosquitos. . .

In [123]:
def mosquito_sequence(day, hop):
    if day == 21:
        return [hop]
    else:
        # Bog 0
        hop = np.random.binomial(1, 0.9)
        if hop == 1:
            print('Day', day, 'Staying in Bog 0')
            mosquito_sequence(day+1, 1)
        # Bog 1
        else:
            print('Day', day, 'Hopped from Bog 0 to Bog 1')
            hop = np.random.binomial(1, 0.8)
            #mosquito_sequence(day+1, 0)
            if hop == 1:
                print('Day', day, 'Staying in Bog 1')
                mosquito_sequence(day+1, 0)
            else:
                print('Day', day, 'Hopping back to Bog 0')
                mosquito_sequence(day+1, 1)

In [124]:
seq = mosquito_sequence(1, 1)

Day 1 Staying in Bog 0
Day 2 Staying in Bog 0
Day 3 Hopped from Bog 0 to Bog 1
Day 3 Staying in Bog 1
Day 4 Staying in Bog 0
Day 5 Staying in Bog 0
Day 6 Staying in Bog 0
Day 7 Staying in Bog 0
Day 8 Staying in Bog 0
Day 9 Staying in Bog 0
Day 10 Staying in Bog 0
Day 11 Staying in Bog 0
Day 12 Staying in Bog 0
Day 13 Staying in Bog 0
Day 14 Staying in Bog 0
Day 15 Hopped from Bog 0 to Bog 1
Day 15 Hopping back to Bog 0
Day 16 Staying in Bog 0
Day 17 Staying in Bog 0
Day 18 Staying in Bog 0
Day 19 Staying in Bog 0
Day 20 Staying in Bog 0
